# 🚁 海南无人机飞行轨迹展示 (修复版)

基于 leafmap GPS Trace 技术，展示海南低空经济中的无人机飞行路径

## 🔧 修复说明

修复了 `add_gps_trace` 方法中缺少 `annotation` 列的问题：
- ❌ 错误: 缺少 `annotation` 列
- ✅ 修复: 添加 `annotation` 列或使用 `ann_column` 参数

## 🎯 应用场景
- 🚁 **无人机巡检路径** - 基础设施巡检飞行轨迹
- 🌾 **农业植保路径** - 农田喷洒作业轨迹
- 📦 **物流配送路径** - 无人机配送飞行轨迹
- 🏖️ **旅游观光路径** - 低空旅游飞行轨迹

In [1]:
# 安装依赖
# %pip install "leafmap[maplibre]" pandas numpy

In [2]:
import leafmap.maplibregl as leafmap
import pandas as pd
import numpy as np
import json
from datetime import datetime, timedelta
import os

print("✅ 库导入完成")

✅ 库导入完成


## 📊 生成海南无人机飞行轨迹数据

In [3]:
def generate_drone_flight_data():
    """生成海南各种无人机飞行轨迹数据"""
    
    # 海口市基础设施巡检路径
    haikou_inspection = {
        "name": "海口电力巡检",
        "type": "基础设施巡检",
        "start_point": [110.32, 20.03],
        "waypoints": [
            [110.32, 20.03],  # 起飞点
            [110.33, 20.04],  # 变电站1
            [110.34, 20.05],  # 输电线路1
            [110.35, 20.04],  # 输电线路2
            [110.36, 20.03],  # 变电站2
            [110.35, 20.02],  # 返回路径
            [110.34, 20.01],  # 检查点
            [110.33, 20.02],  # 最终检查
            [110.32, 20.03],  # 降落点
        ],
        "altitude_range": [50, 120],
        "speed_range": [8, 15],
    }
    
    # 三亚旅游观光路径
    sanya_tourism = {
        "name": "三亚湾观光",
        "type": "旅游观光",
        "start_point": [109.51, 18.25],
        "waypoints": [
            [109.51, 18.25],  # 起飞点(三亚湾)
            [109.50, 18.26],  # 海岸线1
            [109.49, 18.27],  # 海岸线2
            [109.48, 18.28],  # 最西端
            [109.49, 18.29],  # 转向点
            [109.50, 18.28],  # 回程1
            [109.51, 18.27],  # 回程2
            [109.52, 18.26],  # 大东海
            [109.51, 18.25],  # 降落点
        ],
        "altitude_range": [80, 200],
        "speed_range": [5, 12],
    }
    
    # 儋州农业植保路径
    danzhou_agriculture = {
        "name": "儋州农田植保",
        "type": "农业植保",
        "start_point": [109.57, 19.51],
        "waypoints": [
            [109.57, 19.51],  # 起飞点
            [109.58, 19.51],  # 农田区域1
            [109.59, 19.51],  # 农田区域2
            [109.59, 19.52],  # 转向
            [109.58, 19.52],  # 农田区域3
            [109.57, 19.52],  # 农田区域4
            [109.57, 19.53],  # 转向
            [109.58, 19.53],  # 农田区域5
            [109.59, 19.53],  # 农田区域6
            [109.58, 19.52],  # 返回路径
            [109.57, 19.51],  # 降落点
        ],
        "altitude_range": [3, 8],
        "speed_range": [3, 6],
    }
    
    # 琼海物流配送路径
    qionghai_logistics = {
        "name": "琼海城际配送",
        "type": "物流配送",
        "start_point": [110.46, 19.24],
        "waypoints": [
            [110.46, 19.24],  # 配送中心
            [110.47, 19.25],  # 配送点1
            [110.48, 19.26],  # 配送点2
            [110.49, 19.25],  # 配送点3
            [110.48, 19.24],  # 配送点4
            [110.47, 19.23],  # 配送点5
            [110.46, 19.24],  # 返回配送中心
        ],
        "altitude_range": [60, 150],
        "speed_range": [10, 20],
    }
    
    return {
        "海口电力巡检": haikou_inspection,
        "三亚湾观光": sanya_tourism,
        "儋州农田植保": danzhou_agriculture,
        "琼海城际配送": qionghai_logistics,
    }

# 生成飞行数据
drone_flights = generate_drone_flight_data()

print("🚁 海南无人机飞行数据生成完成")
for name, flight in drone_flights.items():
    print(f"   {name}: {flight['type']}, {len(flight['waypoints'])}个航点")

🚁 海南无人机飞行数据生成完成
   海口电力巡检: 基础设施巡检, 9个航点
   三亚湾观光: 旅游观光, 9个航点
   儋州农田植保: 农业植保, 11个航点
   琼海城际配送: 物流配送, 7个航点


## 📝 生成GPS轨迹CSV文件 (修复版)

In [4]:
def create_gps_trace_csv_fixed(flight_data, filename):
    """将飞行数据转换为GPS轨迹CSV格式 - 修复版"""
    
    waypoints = flight_data["waypoints"]
    altitude_range = flight_data["altitude_range"]
    speed_range = flight_data["speed_range"]
    flight_name = flight_data["name"]
    flight_type = flight_data["type"]
    
    # 生成GPS轨迹数据
    gps_data = []
    base_time = datetime.now()
    
    for i, (lon, lat) in enumerate(waypoints):
        # 随机生成高度和速度（在合理范围内）
        altitude = np.random.uniform(altitude_range[0], altitude_range[1])
        speed = np.random.uniform(speed_range[0], speed_range[1])
        
        # 计算时间（基于速度和距离）
        time_offset = timedelta(seconds=i * 30)  # 每30秒一个点
        timestamp = base_time + time_offset
        
        # 创建annotation信息 - 这是关键修复
        annotation = f"{flight_name} - 航点{i+1}: 高度{altitude:.1f}m, 速度{speed:.1f}m/s"
        
        gps_data.append({
            "latitude": lat,
            "longitude": lon,
            "altitude": round(altitude, 1),
            "speed": round(speed, 1),
            "timestamp": timestamp.strftime("%Y-%m-%d %H:%M:%S"),
            "waypoint": i + 1,
            "annotation": annotation,  # 添加annotation列 - 修复关键
            "flight_type": flight_type
        })
    
    # 创建DataFrame并保存为CSV
    df = pd.DataFrame(gps_data)
    
    # 确保data目录存在
    os.makedirs("data", exist_ok=True)
    
    # 保存CSV文件
    csv_path = f"data/{filename}"
    df.to_csv(csv_path, index=False)
    
    print(f"📄 GPS轨迹CSV文件已保存: {csv_path}")
    print(f"   数据点数量: {len(gps_data)}")
    print(f"   飞行时长: {len(gps_data) * 30 / 60:.1f} 分钟")
    print(f"   ✅ 包含annotation列: {df.columns.tolist()}")
    
    return csv_path, df

# 为每个飞行任务生成CSV文件
csv_files = {}
dataframes = {}

for name, flight_data in drone_flights.items():
    filename = f"{name.replace(' ', '_').lower()}_trace.csv"
    csv_path, df = create_gps_trace_csv_fixed(flight_data, filename)
    csv_files[name] = csv_path
    dataframes[name] = df
    print()

print("✅ 所有GPS轨迹CSV文件生成完成 (包含annotation列)")

📄 GPS轨迹CSV文件已保存: data/海口电力巡检_trace.csv
   数据点数量: 9
   飞行时长: 4.5 分钟
   ✅ 包含annotation列: ['latitude', 'longitude', 'altitude', 'speed', 'timestamp', 'waypoint', 'annotation', 'flight_type']

📄 GPS轨迹CSV文件已保存: data/三亚湾观光_trace.csv
   数据点数量: 9
   飞行时长: 4.5 分钟
   ✅ 包含annotation列: ['latitude', 'longitude', 'altitude', 'speed', 'timestamp', 'waypoint', 'annotation', 'flight_type']

📄 GPS轨迹CSV文件已保存: data/儋州农田植保_trace.csv
   数据点数量: 11
   飞行时长: 5.5 分钟
   ✅ 包含annotation列: ['latitude', 'longitude', 'altitude', 'speed', 'timestamp', 'waypoint', 'annotation', 'flight_type']

📄 GPS轨迹CSV文件已保存: data/琼海城际配送_trace.csv
   数据点数量: 7
   飞行时长: 3.5 分钟
   ✅ 包含annotation列: ['latitude', 'longitude', 'altitude', 'speed', 'timestamp', 'waypoint', 'annotation', 'flight_type']

✅ 所有GPS轨迹CSV文件生成完成 (包含annotation列)


## 🗺️ 海口电力巡检无人机轨迹展示 (修复版)

In [5]:
# 创建海口电力巡检轨迹地图
m_haikou = leafmap.Map(
    center=[110.34, 20.03],
    zoom=13,
    pitch=45,
    bearing=0,
    style="basic-v2"
)

# 修复：添加GPS轨迹 - 现在有annotation列了
haikou_csv = csv_files["海口电力巡检"]
m_haikou.add_gps_trace(
    haikou_csv, 
    radius=8,
    add_line=True,
    # 可以指定使用annotation列（默认会自动找到）
    # ann_column="annotation",  # 可选，因为我们已经有annotation列了
    circle_color="red",
    stroke_color="darkred"
)

# 添加标题
m_haikou.add_text(
    "🚁 海口电力巡检无人机轨迹", 
    fontsize=24, 
    padding="15px", 
    position="top-left"
)

# 添加图层控制
m_haikou.add_layer_control()

print("🔌 海口电力巡检无人机轨迹地图创建完成")
m_haikou

🔌 海口电力巡检无人机轨迹地图创建完成


Container(children=[Row(children=[Col(children=[Map(calls=[['addControl', ('NavigationControl', {'showCompass'…

## 🏖️ 三亚旅游观光无人机轨迹展示

In [6]:
# 创建三亚旅游观光轨迹地图
m_sanya = leafmap.Map(
    center=[109.50, 18.27],
    zoom=14,
    pitch=30,
    bearing=-20,
    style="basic-v2"
)

# 添加卫星底图以显示海岸线
m_sanya.add_basemap("Satellite", visible=True, opacity=0.7)

# 添加GPS轨迹
sanya_csv = csv_files["三亚湾观光"]
m_sanya.add_gps_trace(
    sanya_csv, 
    radius=6,
    add_line=True,
    circle_color="blue",
    stroke_color="darkblue"
)

# 添加标题
m_sanya.add_text(
    "🏖️ 三亚湾观光无人机轨迹", 
    fontsize=24, 
    padding="15px", 
    position="top-left"
)

m_sanya.add_layer_control()

print("🌊 三亚旅游观光无人机轨迹地图创建完成")
m_sanya

🌊 三亚旅游观光无人机轨迹地图创建完成


Container(children=[Row(children=[Col(children=[Map(calls=[['addControl', ('NavigationControl', {'showCompass'…

## 🌾 儋州农业植保无人机轨迹展示

In [7]:
# 创建儋州农业植保轨迹地图
m_danzhou = leafmap.Map(
    center=[109.58, 19.52],
    zoom=15,
    pitch=20,
    bearing=0,
    style="basic-v2"
)

# 添加卫星底图以显示农田
m_danzhou.add_basemap("Satellite", visible=True, opacity=0.8)

# 添加GPS轨迹
danzhou_csv = csv_files["儋州农田植保"]
m_danzhou.add_gps_trace(
    danzhou_csv, 
    radius=5,
    add_line=True,
    circle_color="green",
    stroke_color="darkgreen"
)

# 添加标题
m_danzhou.add_text(
    "🌾 儋州农田植保无人机轨迹", 
    fontsize=24, 
    padding="15px", 
    position="top-left"
)

m_danzhou.add_layer_control()

print("🚜 儋州农业植保无人机轨迹地图创建完成")
m_danzhou

🚜 儋州农业植保无人机轨迹地图创建完成


Container(children=[Row(children=[Col(children=[Map(calls=[['addControl', ('NavigationControl', {'showCompass'…

## 📦 琼海物流配送无人机轨迹展示

In [8]:
# 创建琼海物流配送轨迹地图
m_qionghai = leafmap.Map(
    center=[110.47, 19.24],
    zoom=14,
    pitch=35,
    bearing=15,
    style="basic-v2"
)

# 添加GPS轨迹
qionghai_csv = csv_files["琼海城际配送"]
m_qionghai.add_gps_trace(
    qionghai_csv, 
    radius=7,
    add_line=True,
    circle_color="orange",
    stroke_color="darkorange"
)

# 添加标题
m_qionghai.add_text(
    "📦 琼海城际配送无人机轨迹", 
    fontsize=24, 
    padding="15px", 
    position="top-left"
)

m_qionghai.add_layer_control()

print("🚚 琼海物流配送无人机轨迹地图创建完成")
m_qionghai

🚚 琼海物流配送无人机轨迹地图创建完成


Container(children=[Row(children=[Col(children=[Map(calls=[['addControl', ('NavigationControl', {'showCompass'…

## 📋 GPS轨迹数据预览 (修复版)

In [9]:
# 显示修复后的GPS轨迹数据样例
print("📄 修复后的GPS轨迹数据格式示例 (海口电力巡检):")
print("=" * 60)

sample_df = dataframes["海口电力巡检"]
print(sample_df.head(5).to_string(index=False))

print(f"\n📊 数据字段说明:")
print(f"   latitude: 纬度坐标")
print(f"   longitude: 经度坐标")
print(f"   altitude: 飞行高度(米)")
print(f"   speed: 飞行速度(m/s)")
print(f"   timestamp: 时间戳")
print(f"   waypoint: 航点编号")
print(f"   annotation: 注释信息 (修复关键)")
print(f"   flight_type: 飞行任务类型")

print(f"\n✅ 修复验证:")
print(f"   数据列数: {len(sample_df.columns)}")
print(f"   包含annotation列: {'annotation' in sample_df.columns}")
print(f"   数据行数: {len(sample_df)}")

print(f"\n💾 所有CSV文件已保存到 data/ 目录:")
for name, path in csv_files.items():
    print(f"   {path}")

📄 修复后的GPS轨迹数据格式示例 (海口电力巡检):
 latitude  longitude  altitude  speed           timestamp  waypoint                        annotation flight_type
    20.03     110.32      84.0   10.7 2025-07-01 09:21:48         1  海口电力巡检 - 航点1: 高度84.0m, 速度10.7m/s      基础设施巡检
    20.04     110.33     116.6   10.8 2025-07-01 09:22:18         2 海口电力巡检 - 航点2: 高度116.6m, 速度10.8m/s      基础设施巡检
    20.05     110.34      91.2   14.0 2025-07-01 09:22:48         3  海口电力巡检 - 航点3: 高度91.2m, 速度14.0m/s      基础设施巡检
    20.04     110.35      91.8   14.8 2025-07-01 09:23:18         4  海口电力巡检 - 航点4: 高度91.8m, 速度14.8m/s      基础设施巡检
    20.03     110.36      97.1   14.9 2025-07-01 09:23:48         5  海口电力巡检 - 航点5: 高度97.1m, 速度14.9m/s      基础设施巡检

📊 数据字段说明:
   latitude: 纬度坐标
   longitude: 经度坐标
   altitude: 飞行高度(米)
   speed: 飞行速度(m/s)
   timestamp: 时间戳
   waypoint: 航点编号
   annotation: 注释信息 (修复关键)
   flight_type: 飞行任务类型

✅ 修复验证:
   数据列数: 8
   包含annotation列: True
   数据行数: 9

💾 所有CSV文件已保存到 data/ 目录:
   data/海口电力巡检_trace.csv
   data/三

## 🔧 问题修复总结

### ❌ 原始错误
```python
ValueError: Please specify the ann_column parameter or add an 'annotation' column to the GeoDataFrame.
```

### ✅ 修复方案

#### 方案1: 添加annotation列 (采用)
```python
# 在生成CSV数据时添加annotation列
annotation = f"{flight_name} - 航点{i+1}: 高度{altitude:.1f}m, 速度{speed:.1f}m/s"
gps_data.append({
    "latitude": lat,
    "longitude": lon,
    "annotation": annotation,  # 关键修复
    # ... 其他字段
})
```

#### 方案2: 指定ann_column参数 (备选)
```python
# 如果使用其他列作为注释
m.add_gps_trace(csv_file, ann_column="flight_type")
```

### 🎯 修复效果
- ✅ **annotation列**: 包含详细的飞行信息
- ✅ **交互提示**: 点击轨迹点显示注释信息
- ✅ **数据完整**: 所有必需字段都已包含
- ✅ **兼容性**: 符合leafmap GPS trace要求

### 📊 annotation内容示例
```
海口电力巡检 - 航点1: 高度85.3m, 速度12.4m/s
海口电力巡检 - 航点2: 高度92.1m, 速度11.8m/s
三亚湾观光 - 航点1: 高度156.7m, 速度8.2m/s
```

---

**🌴 修复完成！现在可以正常显示海南无人机飞行轨迹了！**

点击地图上的轨迹点可以查看详细的飞行信息！🚁